<a href="https://colab.research.google.com/github/thanksbinh/minBert/blob/main/minBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-v1", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/722k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
ds

Dataset({
    features: ['text'],
    num_rows: 1801350
})

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 512
n_heads = 8 # --> every head is 256/4 = 64 dimensional
n_layers = 6
dropout = 0.1
max_documents = 1801350
# ------------

In [6]:
# prompt: write stoi and itos functions with google-bert/bert-base-cased

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-cased')
tokenizer.add_tokens(['\n'])


def stoi(text):
  """
  String to index.
  Converts a string to a list of token indices using the BERT tokenizer.
  """
  inputs = tokenizer(text, return_tensors="pt").to(device)
  return inputs.input_ids[0]


def itos(token_indices):
  """
  Index to string.
  Converts a list of token indices to a string using the BERT tokenizer.
  """
  # tokens = tokenizer.convert_ids_to_tokens(token_indices)
  # return tokenizer.convert_tokens_to_string(tokens, clean_up_tokenization_spaces=True)
  return tokenizer.decode(token_indices, clean_up_tokenization_spaces=True, skip_special_tokens=True)

def encode(s):
  # return stoi(s)
  sentences = s.split('.')
  encoded_sentences = []
  for sentence in sentences:
    encoded_sentences.append(stoi(sentence))
  # return encoded_sentences
  return torch.cat(encoded_sentences)

def decode(l):
  return itos(l)
  # decoded_sentences = []
  # for sentence in l:
  #   decoded_sentences.append(itos(sentence))
  # return '.'.join(decoded_sentences)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
ds[1]['text']

' = Valkyria Chronicles III = \n'

In [8]:
# create a mapping from characters to integers
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode(ds[0]['text']))
print(decode(encode(ds[0]['text'])))
print(ds[0]['text'] == '')

tensor([101, 102])

True


In [9]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
# data = torch.tensor(encode(text), dtype=torch.long)
data = []
# for i in range(len(ds)):
for i in range(max_documents):
  if ds[i]['text'] == '':
    continue
  data.append(torch.tensor(encode(ds[i]['text'][:512]), dtype=torch.long))
# data = torch.cat(data)
# print(data.shape, data.dtype)
print(len(data))
print(data[:10]) # the 1000 characters we looked at earier will to the GPT look like this

<ipython-input-9-f16b6996443d>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.append(torch.tensor(encode(ds[i]['text']), dtype=torch.long))
Token indices sequence length is longer than the specified maximum sequence length for this model (931 > 512). Running this sequence through the model will result in indexing errors


1165029
[tensor([  101,   134, 12226,  3781,  3464, 17758,  2684,   134, 28996,   102]), tensor([  101, 14895, 21006,  1185, 12226,  3781,  3464,   124,   131,   133,
         8362,  1377,   135, 17758,   113,  1983,   131,   100,   100,   100,
          117,  4941,   102,   101, 12226,  3781,  3464,  1104,  1103,  2651,
         2427,   124,   114,   117,  3337,  2752,  1106,  1112, 12226,  3781,
         3464, 17758,  2684,  1796,  1999,   117,  1110,   170, 12394,  1648,
          137,   118,   137,  1773,  1888,  1342,  1872,  1118, 17979,  1105,
         3957,   102,   101, 11724,  1111,  1103,  8905,  3905,  1895,   102,
          101, 11930,  1107,  1356,  1349,  1107,  1999,   117,  1122,  1110,
         1103,  1503,  1342,  1107,  1103, 12226,  3781,  3464,  1326,   102,
          101, 18653,  1643, 26179,  1158,  1103,  1269, 11970,  1104, 12394,
         1105,  1842,   137,   118,   137,  1159, 11716,  1112,  1157, 17543,
          117,  1103,  1642,  2326,  5504,  1106,  11

In [10]:
vocab_size = 0
for i in range(len(data)):
  vocab_size = max(vocab_size, data[i].max().item())
vocab_size += 1
print(vocab_size)

28997


In [11]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val

train_data = data[:n]
val_data = data[n:]

In [12]:
train_data[0][:block_size+1]

tensor([  101,   134, 12226,  3781,  3464, 17758,  2684,   134, 28996,   102])

In [13]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f"when input is {context} the target: {target}")

In [14]:
from torch.nn import CrossEntropyLoss

def mask_tokens(inputs, mlm_probability=0.15):
    labels = inputs.clone()
    probability_matrix = torch.full(labels.shape, mlm_probability)

    # Create a mask where 15% of tokens will be masked
    mask = torch.bernoulli(probability_matrix).bool()

    # Make sure we don’t mask special tokens like [CLS], [SEP]
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    mask = mask & ~torch.tensor(special_tokens_mask, dtype=torch.bool)

    # 80% of the time, replace masked input tokens with [MASK]
    inputs[mask] = tokenizer.convert_tokens_to_ids("[MASK]")


    # 10% of the time, replace masked input tokens with random words
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    ran = torch.rand(labels.shape) < 0.1
    inputs[mask & ran] = random_words[mask & ran]

    # 10% of the time, keep the original word (no change)

    # Return masked inputs and the labels (for calculating the loss)
    return inputs, labels

def get_batch(split):
    # generate a small batch of data of inputs, labels
    data = train_data if split == 'train' else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))
    batch_data = []
    for i in ix:
      len_batch = 0
      cur_batch = []
      j = i
      while len_batch < block_size:
        if j >= len(data):
          break
        cur_batch.append(data[j])
        len_batch += len(data[j])
        j += 1
      batch_data.append(torch.cat(cur_batch)[:block_size])
    batch_data = torch.stack(batch_data, dim=0)

    inputs, labels = mask_tokens(batch_data)
    inputs, labels = inputs.to(device), labels.to(device)
    return inputs, labels

def get_batch_gpt(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            inputs, labels = get_batch(split)
            logits = model(inputs)

            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
            losses[k] = loss.item()

        out[split] = losses.mean()
    model.train()
    return out

inputs, labels = get_batch('train')
print('inputs:')
print(inputs.shape)
print(inputs[0])
print('labels:')
print(labels.shape)
print(labels[0])

# print('----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([64, 256])
tensor([  101,   134,   134,  1291,  1414,  1563,  1105,  1260,   137,   118,
          103, 13406,   103,   134,   134,   103,   102,   101,  1130,  1347,
         3061,   117,   103,   103,  1105,  1103,  2461,  1913,  1296,  7470,
         1149,  1147, 17074, 23440,   103,  2933,  1107,  2870,  1105,  2882,
         1980,  2258,  1103, 12556,  7841,  3292,   782,   155, 13292, 19145,
        12736, 27175,   102,   101, 20575, 16359,  3950,   103,   137,   117,
          137,  1288,  3129,  3099,  1107,   103,  2638,  3849,  1104,  2870,
         1105,  1107,   103,  3020,   117,  1170,  1199,  3687, 24851,  1891,
          117, 13406,  1105,  1421,  1168,  1484,   103,  1103, 17129,  2875,
        19364,  1186,   117, 12107,  2718,  6582,   103,   117,  1878,  1126,
         1546,  1111,  1147,  7581,  1112,   107, 25170,  1105,   103,  1874,
         6005, 18404,   103,   107,   102,   101,  1332,  1860, 10784,   103,
         2461,  1913,   103,  1340

In [16]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        # compute attention score "affinities"
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)

        # GPT
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        # Todo: check this part again
        # attention_mask = ...
        # wei = wei.masked_fill(attention_mask == 0, float('-inf'))  # Mask padding tokens

        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei) # Prevent some nodes from communicating
        out = wei @ v
        return out

In [17]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [18]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [19]:
class EncoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        # Residual connection, clean flow
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [20]:
class DecoderBlock(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [21]:
class Bert(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[EncoderBlock(n_embed, n_heads=n_heads) for _ in range(n_layers)])
        # self.blocks = nn.Sequential(*[DecoderBlock(n_embed, n_heads=n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        # self.lm_head = nn.Linear(n_embed, vocab_size)
        self.cls = nn.Linear(n_embed, vocab_size) # for MLM

    def forward(self, idx, attention_mask=None, token_type_ids=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        token_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = token_embed + pos_embed # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        # logits = self.lm_head(x) # (B,T,vocab_size)
        logits = self.cls(x)  # (B,T,vocab_size)

        # if targets is None:
        #     loss = None
        # else:
        #     B, T, C = logits.shape
        #     logits = logits.view(B*T, C)
        #     targets = targets.view(B*T)

        #     loss = F.cross_entropy(logits, targets)

        last_hidden_state = x
        return logits, last_hidden_state

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size token
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = Bert()
m = model.to(device)

In [97]:
# xb, yb = get_batch_gpt('train')

# logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)

In [23]:
inputs, labels = get_batch('train')

logits, last_hidden_state = m(inputs)
print(logits.shape)

# Assume `prediction_logits` are the outputs from your model, and `labels` are the true token IDs
loss_fct = CrossEntropyLoss()
masked_lm_loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
print(masked_lm_loss)

torch.Size([64, 256, 28997])
tensor(10.4907, grad_fn=<NllLossBackward0>)


In [24]:
# print(decode(m.generate(idx = torch.tensor([[101]], dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))

In [25]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [26]:
max_iters = 5000
eval_interval = 500

for iter in range(max_iters + 1):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    inputs, labels = get_batch('train')

    # evaluate the loss
    logits, last_hidden_state = m(inputs)
    loss = F.cross_entropy(logits.view(-1, vocab_size), labels.view(-1))

    # zero the gradients from the previous iteration
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 10.4863, val loss 10.4877
step 500: train loss 1.1317, val loss 1.1385
step 1000: train loss 1.0594, val loss 1.0615
step 1500: train loss 0.9625, val loss 0.9631
step 2000: train loss 0.8815, val loss 0.8824
step 2500: train loss 0.8269, val loss 0.8290
step 3000: train loss 0.7852, val loss 0.7870
step 3500: train loss 0.7535, val loss 0.7587
step 4000: train loss 0.7226, val loss 0.7299
step 4500: train loss 0.6999, val loss 0.7077
step 5000: train loss 0.6745, val loss 0.6822



```
step 0: train loss 10.4863, val loss 10.4877
step 500: train loss 1.1317, val loss 1.1385
step 1000: train loss 1.0594, val loss 1.0615
step 1500: train loss 0.9625, val loss 0.9631
step 2000: train loss 0.8815, val loss 0.8824
step 2500: train loss 0.8269, val loss 0.8290
step 3000: train loss 0.7852, val loss 0.7870
step 3500: train loss 0.7535, val loss 0.7587
step 4000: train loss 0.7226, val loss 0.7299
step 4500: train loss 0.6999, val loss 0.7077
step 5000: train loss 0.6745, val loss 0.6822
```

In [27]:
import pickle

In [28]:
filename = 'transformer.sav'
pickle.dump(m, open(filename, 'wb'))

In [29]:
# loading the saved model
loaded_model = pickle.load(open('transformer.sav', 'rb'))

In [30]:
# print(decode(loaded_model.generate(idx = torch.tensor([[ 101 ]], dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))

In [93]:
test_sentence = 'The quick brown fox jumps over the lazy dog'
inputs = stoi(test_sentence)
labels = inputs.clone()
inputs[-3] = 103
inputs = torch.unsqueeze(inputs, dim=0)
labels = torch.unsqueeze(labels, dim=0)

print(inputs)
print(decode(inputs[0]))
print(labels)
print(decode(labels[0]))

logits = m(inputs)
print(logits.shape, logits)

tensor([[  101,  1109,  3613,  3058, 17594, 15457,  1166,  1103,   103,  3676,
           102]])
The quick brown fox jumps over the dog
tensor([[  101,  1109,  3613,  3058, 17594, 15457,  1166,  1103, 16688,  3676,
           102]])
The quick brown fox jumps over the lazy dog
torch.Size([1, 11, 28997]) tensor([[[-7.2175, -5.9410, -7.0223,  ..., -7.9954, -5.4008,  0.4984],
         [-7.0843, -4.8163, -6.4965,  ..., -6.0171, -6.1345,  3.1810],
         [-5.2064, -4.2333, -4.5436,  ..., -3.9867, -3.7042, -0.5569],
         ...,
         [-5.6115, -3.9734, -4.5808,  ..., -4.8803, -4.5746, -0.8368],
         [-4.5261, -3.1427, -4.6268,  ..., -4.8008, -3.3280,  3.7095],
         [-5.3250, -5.8511, -8.0497,  ..., -6.5235, -6.4996,  2.6919]]],
       grad_fn=<ViewBackward0>)


In [94]:
logits_i = logits[:, -3, :] # becomes (B, C)
# apply softmax to get probabilities
probs = F.softmax(logits_i, dim=-1) # (B, C)
# sample from the distribution
print('probs', probs.shape, probs)
# idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
idx_next = torch.argmax(probs, dim=-1) # (B, 1)
# # append sampled index to the running sequence
print('idx_next', idx_next)

probs torch.Size([1, 28997]) tensor([[2.9572e-08, 1.5215e-07, 8.2892e-08,  ..., 6.1434e-08, 8.3401e-08,
         3.5034e-06]], grad_fn=<SoftmaxBackward0>)
idx_next tensor([1514])


In [95]:
decoded_sentences = []
decoded_sentences.append(decode(idx_next))

print(decoded_sentences)

['main']


In [115]:
class BERTForQuestionAnswering(nn.Module):
    def __init__(self):
        super(BERTForQuestionAnswering, self).__init__()

        # Load the BERT model
        self.bert = m

        # Define the start position head
        self.start_head = nn.Linear(vocab_size, 1)

        # Define the end position head
        self.end_head = nn.Linear(vocab_size, 1)

        # Optional: Initialize weights
        nn.init.normal_(self.start_head.weight, std=0.02)
        nn.init.normal_(self.end_head.weight, std=0.02)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Get BERT outputs
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # Get the last hidden state (shape: [batch_size, sequence_length, hidden_size])
        sequence_output = outputs.last_hidden_state
        print(sequence_output.shape)

        # Apply start head (shape: [batch_size, sequence_length, 1])
        start_logits = self.start_head(sequence_output).squeeze(-1)
        print(start_logits.shape)

        # Apply end head (shape: [batch_size, sequence_length, 1])
        end_logits = self.end_head(sequence_output).squeeze(-1)

        return start_logits, end_logits


model_qa = BERTForQuestionAnswering()
m_qa = model_qa.to(device)

In [116]:
inputs, labels = get_batch('train')

start_logits, end_logits = m_qa(inputs)
print(start_logits.shape, end_logits.shape)

# loss_fct = CrossEntropyLoss()
# loss = loss_fct(logits.view(-1, vocab_size), labels.view(-1))
# print(loss)

torch.Size([64, 256, 28997])
torch.Size([64, 256])
torch.Size([64, 256]) torch.Size([64, 256])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BertForQuestionAnswering(nn.Module):
    def __init__(self, bert_model):
        super(BertForQuestionAnswering, self).__init__()
        self.bert = bert_model  # Pretrained BERT model passed as an argument
        hidden_size = bert_model.config.hidden_size  # Hidden size of BERT embeddings

        # Add a linear layer for predicting start and end positions
        self.qa_outputs = nn.Linear(hidden_size, 2)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, start_positions=None, end_positions=None):
        # Forward pass through the BERT model to get hidden states
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs.last_hidden_state  # [batch_size, sequence_length, hidden_size]

        # Apply the linear layer to get start and end logits
        logits = self.qa_outputs(sequence_output)  # [batch_size, sequence_length, 2]
        start_logits, end_logits = logits.split(1, dim=-1)  # Separate start and end logits
        start_logits = start_logits.squeeze(-1)  # [batch_size, sequence_length]
        end_logits = end_logits.squeeze(-1)      # [batch_size, sequence_length]

        # If start_positions and end_positions are provided, compute the loss
        if start_positions is not None and end_positions is not None:
            # Cross-entropy loss for start and end positions
            loss_fct = nn.CrossEntropyLoss()
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2
            return total_loss

        return start_logits, end_logits


In [2]:
import torch
from transformers import BertTokenizerFast, BertModel

# Load pre-trained BERT model and tokenizer for question answering
model_name = "bert-base-uncased"
pretrained_bert = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# model = BertForQuestionAnswering.from_pretrained(model_name)
model = BertForQuestionAnswering(pretrained_bert)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [5]:
from datasets import load_dataset

# Load SQuAD v1.1 dataset
squad = load_dataset("squad")

# Tokenize the inputs for the model
def preprocess_data(batch):
    # Tokenize the batch of questions and contexts
    inputs = tokenizer(
        batch["question"],
        batch["context"],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )
    offset_mapping = inputs.pop("offset_mapping")
    sequence_ids = inputs.sequence_ids()

    # Prepare start and end positions for each example in the batch
    start_positions = []
    end_positions = []

    for i in range(len(batch["answers"])):
        answers = batch["answers"][i]

        # Find start and end character positions
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])

        # Map character positions to token positions
        token_start_index = sequence_ids.index(1)
        token_end_index = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Set token positions
        start_position, end_position = 0, 0
        for j, (offset_start, offset_end) in enumerate(offset_mapping[i]):
            if offset_start <= start_char < offset_end:
                start_position = j
            if offset_start < end_char <= offset_end:
                end_position = j

        start_positions.append(start_position)
        end_positions.append(end_position)

    # Add start and end positions to inputs
    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)

    return inputs


# Apply preprocessing to the train and validation sets
train_data = squad["train"].map(preprocess_data, batched=True, remove_columns=squad["train"].column_names)
val_data = squad["validation"].map(preprocess_data, batched=True, remove_columns=squad["validation"].column_names)


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [39]:
from torch.utils.data import DataLoader, default_collate
from transformers import AdamW

def collate_fn(batch):
    # Stack input_ids, attention_mask, start_positions, and end_positions
    input_ids = torch.stack([torch.tensor(item["input_ids"]) for item in batch])
    attention_mask = torch.stack([torch.tensor(item["attention_mask"]) for item in batch])
    start_positions = torch.stack([torch.tensor(item["start_positions"]) for item in batch])
    end_positions = torch.stack([torch.tensor(item["end_positions"]) for item in batch])

    # Return a dictionary with tensors
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "start_positions": start_positions,
        "end_positions": end_positions
    }


# Define DataLoader
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=8, collate_fn=collate_fn)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].squeeze().to(device)
        attention_mask = batch["attention_mask"].squeeze().to(device)
        start_positions = batch["start_positions"].squeeze().to(device)
        end_positions = batch["end_positions"].squeeze().to(device)

        optimizer.zero_grad()

        # Forward pass
        loss = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        # loss = outputs.total_loss
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/3, Loss: 1.2743


KeyboardInterrupt: 

In [53]:
from transformers import default_data_collator
from tqdm import tqdm
import numpy as np
from evaluate import load

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch["input_ids"].squeeze().to(device)
            attention_mask = batch["attention_mask"].squeeze().to(device)

            # Get model outputs
            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs

            # Predict start and end positions
            start_positions = torch.argmax(start_logits, dim=1)
            end_positions = torch.argmax(end_logits, dim=1)

            # Convert predictions and references to text
            for i in range(len(input_ids)):
                input_id = input_ids[i].tolist()
                start_pos = start_positions[i].item()
                end_pos = end_positions[i].item() + 1  # Inclusive

                # Decode predicted answer
                prediction = tokenizer.decode(input_id[start_pos:end_pos])
                predictions.append(prediction)

                # Reference answer
                answer =tokenizer.decode(input_id[batch["start_positions"][i]:batch["end_positions"][i] + 1])
                references.append(answer)

    # Compute metrics
    squad_metric = load("squad")
    predictions = [{'prediction_text': pred, 'id': str(i)} for i, pred in enumerate(predictions)]
    references = [{'answers': {'answer_start': [ref.index(char) for char in ref], 'text': [ref]}, 'id': str(i)} for i, ref in enumerate(references)]
    results = squad_metric.compute(predictions=predictions, references=references)
    return results

# Run evaluation
scores = evaluate_model(model, val_loader)
print(scores)


100%|██████████| 1322/1322 [01:45<00:00, 12.52it/s]


{'exact_match': 64.24787133396404, 'f1': 78.6242524822625}


In [48]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00


In [49]:
from evaluate import load
squad_metric = load("squad")
predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22'}]
references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
results = squad_metric.compute(predictions=predictions, references=references)
results

{'exact_match': 100.0, 'f1': 100.0}